In [8]:
# Install required libraries (if not already installed)
!pip install requests

# Import libraries
import requests
import sqlite3
from datetime import datetime

# Connect to the database (or create it if it doesn't exist)
db = sqlite3.connect('gists.db')
cursor = db.cursor()

# Create the Gists table if it doesn't exist
cursor.execute("""
CREATE TABLE IF NOT EXISTS gists (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    github_id TEXT NOT NULL,
    html_url TEXT NOT NULL,
    git_pull_url TEXT NOT NULL,
    git_push_url TEXT NOT NULL,
    commits_url TEXT NOT NULL,
    forks_url TEXT NOT NULL,
    public BOOLEAN NOT NULL,
    created_at DATETIME NOT NULL,
    updated_at DATETIME NOT NULL,
    comments INTEGER NOT NULL,
    comments_url TEXT NOT NULL
);
""")
db.commit()

# Function to fetch Gists from GitHub API
def fetch_gists(username):
    url = f"https://api.github.com/users/{username}/gists"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f"Failed to fetch gists: {response.status_code}")

# Function to import Gists into the database
def import_gists_to_database(db, username, commit=True):
    gists = fetch_gists(username)
    cursor = db.cursor()

    for gist in gists:
        cursor.execute("""
            INSERT INTO gists (
                github_id, html_url, git_pull_url, git_push_url, commits_url,
                forks_url, public, created_at, updated_at, comments, comments_url
            ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        """, (
            gist['id'],
            gist['html_url'],
            gist['git_pull_url'],
            gist['git_push_url'],
            gist['commits_url'],
            gist['forks_url'],
            int(gist['public']),  # Convert boolean to integer
            gist['created_at'],
            gist['updated_at'],
            gist['comments'],
            gist['comments_url']
        ))

    if commit:
        db.commit()

# Function to search Gists in the database
def search_gists(db_connection, github_id=None, created_at=None, created_at__gt=None, created_at__gte=None,
                 created_at__lt=None, created_at__lte=None, updated_at__gt=None, updated_at__gte=None,
                 updated_at__lt=None, updated_at__lte=None):
    query = "SELECT * FROM gists WHERE 1=1"
    params = {}

    if github_id:
        query += " AND github_id = :github_id"
        params['github_id'] = github_id

    if created_at:
        query += " AND datetime(created_at) = datetime(:created_at)"
        params['created_at'] = created_at

    if created_at__gt:
        query += " AND datetime(created_at) > datetime(:created_at__gt)"
        params['created_at__gt'] = created_at__gt

    if created_at__gte:
        query += " AND datetime(created_at) >= datetime(:created_at__gte)"
        params['created_at__gte'] = created_at__gte

    if created_at__lt:
        query += " AND datetime(created_at) < datetime(:created_at__lt)"
        params['created_at__lt'] = created_at__lt

    if created_at__lte:
        query += " AND datetime(created_at) <= datetime(:created_at__lte)"
        params['created_at__lte'] = created_at__lte

    if updated_at__gt:
        query += " AND datetime(updated_at) > datetime(:updated_at__gt)"
        params['updated_at__gt'] = updated_at__gt

    if updated_at__gte:
        query += " AND datetime(updated_at) >= datetime(:updated_at__gte)"
        params['updated_at__gte'] = updated_at__gte

    if updated_at__lt:
        query += " AND datetime(updated_at) < datetime(:updated_at__lt)"
        params['updated_at__lt'] = updated_at__lt

    if updated_at__lte:
        query += " AND datetime(updated_at) <= datetime(:updated_at__lte)"
        params['updated_at__lte'] = updated_at__lte

    cursor = db_connection.execute(query, params)
    return cursor.fetchall()

# Example usage
try:
    # Import Gists for the user 'gvanrossum'
    import_gists_to_database(db, 'gvanrossum')

    # Search for Gists created after January 1, 2018
    results = search_gists(db, created_at__gt='2018-01-01T00:00:00Z')
    for row in results:
        print(row)

finally:
    # Close the database connection
    db.close()

(1, 'a465d31d9402bae2c79e89b2f344c10c', 'https://gist.github.com/gvanrossum/a465d31d9402bae2c79e89b2f344c10c', 'https://gist.github.com/a465d31d9402bae2c79e89b2f344c10c.git', 'https://gist.github.com/a465d31d9402bae2c79e89b2f344c10c.git', 'https://api.github.com/gists/a465d31d9402bae2c79e89b2f344c10c/commits', 'https://api.github.com/gists/a465d31d9402bae2c79e89b2f344c10c/forks', 1, '2022-05-03T19:59:29Z', '2022-05-09T03:14:49Z', 1, 'https://api.github.com/gists/a465d31d9402bae2c79e89b2f344c10c/comments')
(2, 'b8ca09175a0d1399a8999f13c7bfa616', 'https://gist.github.com/gvanrossum/b8ca09175a0d1399a8999f13c7bfa616', 'https://gist.github.com/b8ca09175a0d1399a8999f13c7bfa616.git', 'https://gist.github.com/b8ca09175a0d1399a8999f13c7bfa616.git', 'https://api.github.com/gists/b8ca09175a0d1399a8999f13c7bfa616/commits', 'https://api.github.com/gists/b8ca09175a0d1399a8999f13c7bfa616/forks', 1, '2021-09-21T19:25:46Z', '2021-09-21T19:54:25Z', 0, 'https://api.github.com/gists/b8ca09175a0d1399a8999f